In [1]:
import os
import sys
import pdb
import rospy
import numpy as np
import torch
from visualization_msgs.msg import *

# Adds parent dir to path
current_dir = os.getcwd()
base_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.insert(0, base_dir)

from Models.DiscreteBKI import *
from Data.dataset import Rellis3dDataset, ray_trace_batch
from Data.utils import *
from model_utils import *
from sklearn.metrics import jaccard_score

TIME_MEASUREMENTS=False
NUM_CLASSES=21
DATASET_DIR = "/home/arthurzhang/Data/Rellis-3D"
sbki_folder = "/home/arthurzhang/CURLY/Baselines/catkin_ws/src/BKISemanticMapping/data/rellis3d/test"


In [1]:
gt_all = np.array([])
pred_all = np.array([])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

rospy.init_node('talker', anonymous=True)
map_pub = rospy.Publisher('SemMap', MarkerArray, queue_size=10)
bki_map = DiscreteBKI(
    torch.tensor([256, 256, 16]).to(device), # Grid size
    torch.tensor([-25.6, -25.6, -2.0]).to(device), # Lower bound
    torch.tensor([25.6, 25.6, 1.2]).to(device), # Upper bound
    device=device
)

rellis_ds = Rellis3dDataset(directory=DATASET_DIR, device=device, 
    num_frames=10, remap=True, use_aug=False, use_gt=True, model_setting="test")


num_correct = 0
num_total = 0
# running_inter = np.zeros((NUM_CLASSES,))
# running_union =  np.zeros((NUM_CLASSES,))
running_inter = np.zeros((NUM_CLASSES,))
running_union = np.zeros((NUM_CLASSES,)) + 1e-6
running_ssc_inter = np.zeros(NUM_CLASSES,)
running_ssc_union = np.zeros(NUM_CLASSES,) + 1e-6
running_geo_score = np.zeros((2,))
running_counter = 0
curr_time = time.time()
for idx in range(1, len(rellis_ds)):
    sem_evaluation_dir = os.path.join(sbki_folder, "evaluations", str(idx).zfill(6)+".txt")
    ssc_evaluation_dir = os.path.join(sbki_folder, "ssc", str(idx).zfill(6)+".txt")
    
    if TIME_MEASUREMENTS: curr_time = time.time()
    sem_evaluation = np.fromfile(sem_evaluation_dir, sep="\n", dtype=np.uint8).reshape(-1)
    ssc_evaluation = np.fromfile(ssc_evaluation_dir, sep="\n",dtype=np.uint8)

    if TIME_MEASUREMENTS: 
        print("time to load files ", time.time() - curr_time)
        curr_time = time.time()

    # Load rellis3D dataset into np arrays
    gt_pc, gt_labels, gt_voxels_np, _, _ = rellis_ds[idx]
    gt_pc       = np.vstack(np.array(gt_pc))
    gt_labels   = np.vstack(np.array(gt_labels))
    gt_voxels_np = gt_voxels_np.reshape(-1)
    if TIME_MEASUREMENTS:
        print("time to load rellis ", time.time() - curr_time)
        curr_time = time.time()

    # Process free space labels
    fs_pc           = ray_trace_batch(gt_pc, gt_labels, 0.3, device)
    gt_pc_np        = np.vstack( (gt_pc, fs_pc[:, :3].reshape(-1, 3)))
    gt_labels_np    = np.vstack( (gt_labels, fs_pc[:, 3].reshape(-1, 1))).reshape(-1)
    if TIME_MEASUREMENTS:
        print("time to raytrace ", time.time() - curr_time)
        curr_time = time.time()

    # Ignore void labels from ground truth
    void_mask = gt_labels_np!=0
    gt_pc_np = gt_pc_np[void_mask]
    gt_labels_np = gt_labels_np[void_mask]

    if gt_pc_np.shape[0] <= 0:
        # Zero pad in case all labels are 0
        gt_pc_np = np.zeros((1, 3))
        gt_labels_np = np.zeros((1,))

    sem_evaluation_torch = torch.from_numpy(sem_evaluation)
    gt_labels_torch      = torch.from_numpy(gt_labels_np)
    ssc_evaluation_torch = torch.from_numpy(ssc_evaluation)
    gt_voxels_torch      = torch.from_numpy(gt_voxels_np)
    
    if TIME_MEASUREMENTS:
        print("time to compute iou ", time.time() - curr_time)
    inter, union = iou_one_frame(sem_evaluation_torch, gt_labels_torch, n_classes=NUM_CLASSES)
    # iou_score = jaccard_score(gt_labels_np, sem_evaluation, 
    #     labels=np.arange(0, NUM_CLASSES), average=None, zero_division=0)

    ssc_inter, ssc_union = iou_one_frame(ssc_evaluation_torch, gt_voxels_torch, n_classes=NUM_CLASSES)
    # jaccard_score(gt_voxels_np, ssc_evaluation, 
    #     labels=np.arange(0, NUM_CLASSES), average=None, zero_division=0)
    running_inter   += inter
    running_union   += union
    running_ssc_inter   += ssc_inter
    running_ssc_union   += ssc_union
    _, _, iou = geo_complete_score(gt_voxels_np, ssc_evaluation, LABELS_REMAP[-1])
    running_geo_score += iou

    running_counter += 1
    num_correct += np.sum(sem_evaluation==gt_labels_np)
    num_total   += gt_labels_np.shape[0]

    # running_union[running_union==0] = 1
    # running_iou = (running_inter / running_union)
    if running_counter%20==0:
        print("mIoU ", np.mean(running_inter/running_union))
        print("SSC mIoU ", np.mean(running_ssc_inter / running_ssc_union))
        print("Geometric Completeness ", (running_geo_score / running_counter))
        print("Accuracy ", num_correct / num_total)
        print("Time for iteration ", running_counter, " is ", time.time() - curr_time)
    
    curr_time = time.time()
    # Visualization for Debugging
    # gt_pc_torch = torch.from_numpy(gt_pc_np)
    # preds = torch.from_numpy(sem_evaluation.astyspe(np.uint8)).reshape(-1)
    # publish_pc(gt_pc_torch, preds, map_pub, 
    #         bki_map.min_bound.reshape(-1),
    #         bki_map.max_bound.reshape(-1),
    #         bki_map.grid_size.reshape(-1))
    # pdb.set_trace()
    # gt_labels_torch = torch.from_numpy(gt_labels_np.astype(np.uint8)).reshape(-1)
    # publish_pc(gt_pc_torch, gt_labels_torch, map_pub, 
    #         bki_map.min_bound.reshape(-1),
    #         bki_map.max_bound.reshape(-1),
    #         bki_map.grid_size.reshape(-1))
    # pdb.set_trace()

print("Final mIoU ", np.mean(running_inter / running_union))
print("Final SSC mIoU ", np.mean(running_ssc_inter / running_ssc_union))
print("Final IoU ", running_inter / running_union)
print("Final SSC IoU ", running_ssc_inter / running_ssc_union)
print("Final Geometric Completeness ", (running_geo_score / running_counter))
print("Final Accuracy ", num_correct / num_total)



mIoU  0.2595661615785413
SSC mIoU  0.04215192392227252
Geometric Completeness  [0.20323419 0.68672557]
Accuracy  0.9766623297000642
Time for iteration  20  is  15.297344446182251
mIoU  0.26049211257869
SSC mIoU  0.042505578454149685
Geometric Completeness  [0.20761659 0.68631093]
Accuracy  0.9767574815824979
Time for iteration  40  is  14.908189058303833
mIoU  0.26124320066529017
SSC mIoU  0.042530556319581025
Geometric Completeness  [0.20834141 0.68567126]
Accuracy  0.9768424953483864
Time for iteration  60  is  15.624265670776367
mIoU  0.2606547833447172
SSC mIoU  0.042420947661538845
Geometric Completeness  [0.20930839 0.68307165]
Accuracy  0.9769132381638244
Time for iteration  80  is  15.373626470565796
mIoU  0.25971625248604024
SSC mIoU  0.04231245807705872
Geometric Completeness  [0.21203383 0.67658194]
Accuracy  0.9767296135752828
Time for iteration  100  is  15.346311092376709
mIoU  0.2532017620544916
SSC mIoU  0.04216529594341651
Geometric Completeness  [0.21123654 0.66995941

In [2]:
print("Final mIoU ", np.mean(running_iou / running_counter))
print("Final SSC mIoU ", np.mean(running_ssc_iou / running_counter))
print("Final IoU ", running_iou / running_counter)
print("Final SSC IoU ", running_ssc_iou / running_counter)
print("Final Geometric Completeness ", (running_geo_score / running_counter))
print("Final Accuracy ", num_correct / num_total)

Final mIoU  0.16358611325545627
Final SSC mIoU  0.035134386534124235
Final IoU  [0.00000000e+00 0.00000000e+00 5.60016396e-01 5.88545616e-01
 1.43518530e-01 9.22742464e-06 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.62998573e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.06223123e-02 5.21189291e-01 3.04980999e-01
 7.74924694e-02 1.03251444e-01 1.13727678e-01 2.97595047e-03
 9.56348479e-01]
Final SSC IoU  [6.70416552e-01 0.00000000e+00 8.29817942e-03 5.21273223e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 7.47226060e-06 1.41210174e-03 1.08312520e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.60834131e-02]
Final Geometric Completeness  [0.05608341 0.69161202]
Final Accuracy  0.9584161341756463


In [4]:

final_iou_scores = np.array([
    0.00000000e+00,
    0.00000000e+00,
    5.60016396e-01,
    5.88545616e-01,
    1.43518530e-01,
    9.22742464e-06,
    0.00000000e+00,
    0.00000000e+00,
    0.00000000e+00,
    2.62998573e-03,
    0.00000000e+00,
    0.00000000e+00,
    0.00000000e+00,
    6.06223123e-02,
    5.21189291e-01,
    3.04980999e-01,
    7.74924694e-02,
    1.03251444e-01,
    1.13727678e-01,
    2.97595047e-03,
    9.56348479e-01])

final_ssc_iou_scores = np.array([
    6.70416552e-01,
    0.00000000e+00, 
    8.29817942e-03, 
    5.21273223e-04,
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00,
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00,
    0.00000000e+00, 
    7.47226060e-06, 
    1.41210174e-03, 
    1.08312520e-03,
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00, 
    0.00000000e+00,
    5.60834131e-02
])

# print("Number correct ", num_correct, " number total ", num_total)
print("mIOU with empty classes removed ", np.mean(final_iou_scores[CLASS_COUNTS_REMAPPED>0]))
print("ssc IOU with empty classes removed ", np.mean(final_ssc_iou_scores[CLASS_COUNTS_REMAPPED>0]))

mIOU with empty classes removed  0.24537916988033145
ssc IOU with empty classes removed  0.0048146832102571435
